# Homework 3: Mining Data Streams
#### Authors: Sherly Sherly and Anna Martignano

## Task
You are to study and implement a streaming graph processing algorithm described in one of the above papers of your choice. In order to accomplished your task, you are to perform the following two steps

1. Implement the reservoir sampling or the Flajolet-Martin algorithm used in the graph algorithm presented in the paper you have selected;
2. Implement the streaming graph algorithm presented in the paper that make use of the algorithm implemented in the first step. 

## Reservoir sampling in TRIEST-BASE
The trièst-base algorithm uses standard reservoir sampling to maintain the edge sample S:
- If t ≤ M, then the edge et = (u, v) on the stream at time t is deterministically inserted in S.
- If t > M, trièst-base flips a biased coin with heads probability M/t. If the outcome is heads, it chooses an edge (w, z) ∈ S uniformly at random, removes (w, z) from S, and inserts (u, v) in S. Otherwise, S is not modified.

        Input: Insertion-only edge stream Σ, integer M ≥ 6
        S ← ∅, t ← 0, τ ← 0
        for each element (+, (u, v)) from Σ do
            t ← t + 1
            if SampleEdge((u, v), t) then
                S ← S ∪ {(u, v)}
                UpdateCounters(+, (u, v))

        function SampleEdge((u, v), t)
            if t ≤ M then
                return True
            else if FlipBiasedCoin(M/t) = heads then
                (u', v') ← random edge from S
                S ← S \ {(u', v')}
                UpdateCounters(−, (u',v'))
                return True
            return False

        function UpdateCounters((•, (u, v)))
            NSu,v ← NSu ∩ NSv
            for all c ∈ NSu,v do
                τ ← τ • 1
                τc ← τc • 1
                τu ← τu • 1
                τv ← τv • 1


    

In [1]:
from collections import defaultdict
import random
import operator

class Graph:
    def __init__(self, M):
        self.M = M
        self.t = 0
        self.sampleSet = set()
        self.dictNeighbor = {}
        self.tau = defaultdict(lambda:0)
        self.TAU = 0
        
        
    def addEdge(self, u, v):
        self.sampleSet.add((u,v))
            
        if u in self.dictNeighbor.keys():
            self.dictNeighbor[u].add(v)
        else:
            self.dictNeighbor[u] = set([v])

        if v in self.dictNeighbor.keys():
            self.dictNeighbor[v].add(u)
        else:
            self.dictNeighbor[v] = set([u])
    
    def removeEdge(self, w, x):
        self.sampleSet.remove((w,x))
            
        self.dictNeighbor[w].remove(x)
        self.dictNeighbor[x].remove(w)
            
        if(len(self.dictNeighbor[w]) == 0):
            del self.dictNeighbor[w]
        if(len(self.dictNeighbor[x]) == 0):
            del self.dictNeighbor[x]
                
    def sampleEdge(self):
        if(self.t <= self.M):
            return True
        else:
            flipCoin = random.random()
            if(flipCoin < self.M/float(self.t)):
                edge = random.choice(tuple(self.sampleSet))
                w = int(edge[0])
                x = int(edge[1])
                self.removeEdge(w,x)
                self.updateCounters("-",w,x)
                return True
        return False
    
    def updateCounters(self,operation,u,v):
        if(u in self.dictNeighbor.keys() and v in self.dictNeighbor.keys()):
            NSu = self.dictNeighbor[u]
            NSv = self.dictNeighbor[v]

            NSuv = NSu & NSv

            ops = { "+": operator.add, "-": operator.sub }        
            if(len(NSuv) > 0):
                for c in NSuv:
                    self.tau[c] = ops[operation](self.tau[c],1)
                    self.tau[u] = ops[operation](self.tau[u],1)
                    self.tau[v] = ops[operation](self.tau[v],1)
                    self.TAU = ops[operation](self.TAU,1)
                
    def launchTriest(self,filepath):
        file = open(filepath,'r')
        for line in file:
            input = line.split()
            u = int(min(input))
            v = int(max(input))
            self.t += 1
            
            if((u,v) in self.sampleSet or u == v):
                continue
            
            if(self.sampleEdge()):
                self.addEdge(u,v)
                self.updateCounters("+",u,v)
            
            if(self.t%10000==0):
                print("Iteration {}|Global TAU {}|Samples {}".format(self.t,self.TAU,len(self.sampleSet)))
        file.close() 

In [2]:
triestBase = Graph(10000)
triestBase.launchTriest('com-amazon.tar/com-amazon.tar')

Iteration 10000|Global TAU 3009|Samples 10000
Iteration 20000|Global TAU 817|Samples 10000
Iteration 30000|Global TAU 386|Samples 10000
Iteration 40000|Global TAU 226|Samples 10000
Iteration 50000|Global TAU 138|Samples 10000
Iteration 60000|Global TAU 100|Samples 10000
Iteration 70000|Global TAU 76|Samples 10000
Iteration 80000|Global TAU 54|Samples 10000
Iteration 90000|Global TAU 48|Samples 10000
Iteration 100000|Global TAU 37|Samples 10000
Iteration 110000|Global TAU 31|Samples 10000
Iteration 120000|Global TAU 25|Samples 10000
Iteration 130000|Global TAU 21|Samples 10000
Iteration 140000|Global TAU 22|Samples 10000
Iteration 150000|Global TAU 27|Samples 10000
Iteration 160000|Global TAU 19|Samples 10000
Iteration 170000|Global TAU 18|Samples 10000
Iteration 180000|Global TAU 15|Samples 10000
Iteration 190000|Global TAU 10|Samples 10000
Iteration 200000|Global TAU 10|Samples 10000
Iteration 210000|Global TAU 11|Samples 10000
Iteration 220000|Global TAU 9|Samples 10000
Iteration 230

## Reservoir sampling in TRIEST-IMPR
        Input: Insertion-only edge stream Σ, integer M ≥ 6
        S ← ∅, t ← 0, τ ← 0
        for each element (+, (u, v)) from Σ do
            t ← t + 1
            UpdateCounters((u, v))
            if SampleEdge((u, v), t) then
                S ← S ∪ {(u, v)}

        function SampleEdge((u, v), t)
            if t ≤ M then
                return True
            else if FlipBiasedCoin(M/t) = heads then
                (u', v') ← random edge from S
                S ← S \ {(u', v')}
                return True
            return False

        function UpdateCounters((u, v))
            NSu,v ← NSu ∩ NSv
            η(t) = max{1,(t − 1)(t − 2)/(M(M − 1))}
            for all c ∈ NSu,v do
                τ ← τ + η(t)
                τc ← τc + η(t)
                τu ← τu + η(t)
                τv ← τv + η(t)

In [3]:
from collections import defaultdict
import random
import operator

class GraphImpr:
    def __init__(self, M):
        self.M = M
        self.t = 0
        self.sampleSet = set()
        self.dictNeighbor = {}
        self.tau = defaultdict(lambda:0)
        self.TAU = 0
        
        
    def addEdge(self, u, v):
        self.sampleSet.add((u,v))
            
        if u in self.dictNeighbor.keys():
            self.dictNeighbor[u].add(v)
        else:
            self.dictNeighbor[u] = set([v])

        if v in self.dictNeighbor.keys():
            self.dictNeighbor[v].add(u)
        else:
            self.dictNeighbor[v] = set([u])
    
    def removeEdge(self, w, x):
        self.sampleSet.remove((w,x))
            
        self.dictNeighbor[w].remove(x)
        self.dictNeighbor[x].remove(w)
            
        if(len(self.dictNeighbor[w]) == 0):
            del self.dictNeighbor[w]
        if(len(self.dictNeighbor[x]) == 0):
            del self.dictNeighbor[x]
                
    def sampleEdge(self):
        if(self.t <= self.M):
            return True
        else:
            flipCoin = random.random()
            if(flipCoin < self.M/float(self.t)):
                edge = random.choice(tuple(self.sampleSet))
                w = int(edge[0])
                x = int(edge[1])
                self.removeEdge(w,x)
                return True
        return False
    
    def updateCounters(self,u,v):
        if(u in self.dictNeighbor.keys() and v in self.dictNeighbor.keys()):
            NSu = self.dictNeighbor[u]
            NSv = self.dictNeighbor[v]

            NSuv = NSu & NSv
            
            eta = max(1, ((self.t-1)*(self.t-2)/(self.M*(self.M-1))) )
      
            if(len(NSuv) > 0):
                for c in NSuv:
                    self.tau[c] += eta
                    self.tau[u] += eta
                    self.tau[v] += eta
                    self.TAU += eta
                
    def launchTriest(self,filepath):
        file = open(filepath,'r')
        for line in file:
            input = line.split()
            u = int(min(input))
            v = int(max(input))
            self.t += 1
            
            if((u,v) in self.sampleSet or u == v):
                continue
            
            self.updateCounters(u,v)
            
            if(self.sampleEdge()):
                self.addEdge(u,v)
            
            if(self.t%10000==0):
                print("Iteration {}|Global TAU {}|Samples {}".format(self.t,self.TAU,len(self.sampleSet)))
        file.close() 

In [4]:
triestImpr = GraphImpr(10000)
triestImpr.launchTriest('com-amazon.tar/com-amazon.tar')

Iteration 10000|Global TAU 3009|Samples 10000
Iteration 20000|Global TAU 6646.67917505751|Samples 10000
Iteration 30000|Global TAU 11096.906209400924|Samples 10000
Iteration 40000|Global TAU 15322.554175977573|Samples 10000
Iteration 50000|Global TAU 19417.550387158688|Samples 10000
Iteration 60000|Global TAU 23382.822722392208|Samples 10000
Iteration 70000|Global TAU 27307.762178417797|Samples 10000
Iteration 80000|Global TAU 32237.287185438483|Samples 10000
Iteration 90000|Global TAU 37504.873246724615|Samples 10000
Iteration 100000|Global TAU 41985.278543814326|Samples 10000
Iteration 110000|Global TAU 45603.18430481044|Samples 10000
Iteration 120000|Global TAU 50420.0918366236|Samples 10000
Iteration 130000|Global TAU 57186.64961506144|Samples 10000
Iteration 140000|Global TAU 63064.60339709963|Samples 10000
Iteration 150000|Global TAU 66852.95246722664|Samples 10000
Iteration 160000|Global TAU 72157.58860224014|Samples 10000
Iteration 170000|Global TAU 76827.09838233818|Samples 10